In [1]:
# == importar bibliotecas == 

from bs4 import BeautifulSoup as bs
import unicodedata as ucd
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
import os
from itertools import chain 
from selenium import webdriver
from selenium.webdriver.common.by import By

In [62]:
# == settando variáveis == 

JORNAL = 'NOVO'
LABEL = 'direita' 
URL = "https://novo.org.br/noticias/"
REF= 50
str_load = 'setOffset(1)'
class_1 = 'astute-single-blog'
class_2 = 'single-blog-content'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

In [19]:
# == configurando env == 

driver= webdriver.Chrome()
driver.get(URL)
driver.maximize_window()

In [41]:
# == obter html == 

def get_soup(url):
    response = requests.get(url, headers=headers, allow_redirects=False)
    html = response.content
    soup = bs(html, 'html.parser')
    return soup

In [21]:
# == obter dataframe do texto == 

def return_df(url, text):
    return pd.DataFrame({
        'jornal': JORNAL,
        'url': url,
        'artigo': text,
        'label': LABEL
    }, index=[1])

In [22]:
# == coletar o site dos artigos == 

def get_links(driver):
    cur_list = []
    soup = bs(driver.page_source, 'lxml')
    for tag in soup.find_all("a", href=True, class_=class_1):
        cur_list.append(tag.get('href'))
    return cur_list

In [23]:
# == obter conteúdo == 

list_url = []
for n in range(REF):
    list_url.append(get_links(driver))
    but = driver.find_element(By.XPATH, f"//button[contains(@onclick,'{str_load}')]")
    driver.execute_script("arguments[0].click();", but)
    sleep(5)


100%|██████████| 50/50 [05:45<00:00,  6.91s/it]


In [25]:
# == remover url duplicado == 

list_url = list(chain.from_iterable(list_url))
list_url = list(set(list_url))

In [ ]:
# == obter conteúdo da notícia == 

def get_artigo(url):
    soup = get_soup(url)
    list_text = []
    for div in soup.find_all("div", class_=class_2):
        for p in div.find_all("p"):
            list_text.append(ucd.normalize("NFKD", p.text))
    return return_df(url, " ".join(list_text))

res = []
for url in list_url:
    res.append(get_artigo(url))
scrap = pd.concat(res)

In [70]:
# == salvar o scrapping == 

scrap.to_parquet('../dataset/scrapping/NOVO.parquet')
scrap.to_csv('../dataset/scrapping/NOVO.csv')

In [71]:
scrap.shape

(1198, 4)